# Final Project

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

#### Import data

In [3]:
Monday_df = pd.read_csv('archive_2/Monday_WorkingHours.csv')
Monday_df = pd.DataFrame(Monday_df)
print(Monday_df.shape) 
Monday_df.columns = Monday_df.columns.str.strip() #Remove extra spaces from column names

Tuesday_BruteForce = pd.read_csv('archive_2/Tuesday_BruteForce.csv')
Tuesday_BruteForce = pd.DataFrame(Tuesday_BruteForce)
print(Tuesday_BruteForce.shape)
Tuesday_BruteForce.columns = Tuesday_BruteForce.columns.str.strip()

Wednesday_DOS = pd.read_csv('archive_2/Wednesday_DOS_DDOS.csv')
Wednesday_DOS = pd.DataFrame(Wednesday_DOS)
print(Wednesday_DOS.shape)
Wednesday_DOS.columns = Wednesday_DOS.columns.str.strip() 

Thursday_WebAttack = pd.read_csv('archive_2/Thursday_Morning_WebAttacks.csv')
Thursday_WebAttack = pd.DataFrame(Thursday_WebAttack)
print(Thursday_WebAttack.shape)
Thursday_WebAttack.columns = Thursday_WebAttack.columns.str.strip()

Thursday_Infiltration = pd.read_csv('archive_2/Thursday_Afternoon_Infiltration.csv')
Thursday_Infiltration = pd.DataFrame(Thursday_Infiltration)
print(Thursday_Infiltration.shape)
Thursday_Infiltration.columns = Thursday_Infiltration.columns.str.strip()

Friday_Botnet = pd.read_csv('archive_2/Friday_Morning_Botnet.csv')
Friday_Botnet = pd.DataFrame(Friday_Botnet)
print(Friday_Botnet.shape)
Friday_Botnet.columns = Friday_Botnet.columns.str.strip()

Friday_Portscan = pd.read_csv('archive_2/Friday_Afternoon_PortScan.csv')
Friday_Portscan = pd.DataFrame(Friday_Portscan)
print(Friday_Portscan.shape)
Friday_Portscan.columns = Friday_Portscan.columns.str.strip()

Friday_DDOS = pd.read_csv('archive_2/Friday_Afternoon_DDos.csv')
Friday_DDOS = pd.DataFrame(Friday_DDOS)
print(Friday_DDOS.shape)
Friday_DDOS.columns = Friday_DDOS.columns.str.strip()

(529918, 79)
(445909, 79)
(692703, 79)
(170366, 79)
(288602, 79)
(191033, 79)
(286467, 79)
(225745, 79)


#### Explore columns to be removed

In [4]:
def analyzeFeatures(df):
    results = {}
    # Duplicate columns
    checked = []
    duplicate_cols = []
    for col1 in df.columns:
        for col2 in df.columns:
            if col1 != col2 and col2 not in checked:
                if df[col1].equals(df[col2]):
                    duplicate_cols.append((col1,col2))
                    checked.append(col2)
    results['duplicates'] = duplicate_cols

    #Not unique columns
    constant_cols = [col for col in df.columns if df[col].nunique() == 1]
    results['constant_cols'] = constant_cols

    #Missing value columns
    missing = df.isna().mean().sort_values(ascending=False)
    results['missing'] = missing[missing > 0].to_dict()

    return results

Mon_features = analyzeFeatures(Monday_df)
BruteForce_features = analyzeFeatures(Tuesday_BruteForce)
DOS_features = analyzeFeatures(Wednesday_DOS)
WebAttack_features = analyzeFeatures(Thursday_WebAttack)
Infiltration_features = analyzeFeatures(Thursday_Infiltration)
Botnet_features = analyzeFeatures(Friday_Botnet)
Portscan_features = analyzeFeatures(Friday_Portscan)
DDOS_features = analyzeFeatures(Friday_DDOS)

dataset_features = [Mon_features, BruteForce_features, DOS_features, WebAttack_features, Infiltration_features, Botnet_features, Portscan_features, DDOS_features]

def feature_summary(dataset_features):
    duplicate_lists = [set(map(tuple, d['duplicates'])) for d in dataset_features]
    constant_lists = [set(d['constant_cols']) for d in dataset_features]
    missing_lists = [set(d['missing'].keys()) for d in dataset_features]

    common_duplicates = set.intersection(*duplicate_lists) if duplicate_lists else set()
    common_constants = set.intersection(*constant_lists) if constant_lists else set()
    common_missing = set.intersection(*missing_lists) if missing_lists else set()

    return {
        'common_duplicate': sorted(list(common_duplicates)),
        'common_constant': sorted(list(common_constants)),
        'common_missing': sorted(list(common_missing))
    }

summary = feature_summary(dataset_features)
print(summary)

{'common_duplicate': [('Bwd PSH Flags', 'Bwd Avg Bulk Rate'), ('Bwd PSH Flags', 'Bwd Avg Bytes/Bulk'), ('Bwd PSH Flags', 'Bwd Avg Packets/Bulk'), ('Bwd PSH Flags', 'Bwd URG Flags'), ('Bwd PSH Flags', 'Fwd Avg Bulk Rate'), ('Bwd PSH Flags', 'Fwd Avg Bytes/Bulk'), ('Bwd PSH Flags', 'Fwd Avg Packets/Bulk'), ('Fwd Header Length', 'Fwd Header Length.1'), ('Fwd Header Length.1', 'Fwd Header Length'), ('Fwd PSH Flags', 'SYN Flag Count'), ('SYN Flag Count', 'Fwd PSH Flags'), ('Subflow Bwd Packets', 'Total Backward Packets'), ('Subflow Fwd Packets', 'Total Fwd Packets'), ('Total Backward Packets', 'Subflow Bwd Packets'), ('Total Fwd Packets', 'Subflow Fwd Packets')], 'common_constant': ['Bwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd PSH Flags', 'Bwd URG Flags', 'Fwd Avg Bulk Rate', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk'], 'common_missing': ['Flow Bytes/s']}


#### Drop columns and create y dataframe

In [5]:
def copyLabelDropColumns(df):
    yDF = df[['Label']].copy()
    newDF = df.drop(['Destination Port', 'Label', 'Bwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd PSH Flags', 'Bwd URG Flags', 'Fwd Avg Bulk Rate', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Header Length.1', 'Fwd PSH Flags', 'Subflow Bwd Packets','Subflow Fwd Packets'], axis=1)
    return yDF, newDF

In [6]:
#Monday
Mon_df_y, Mon_drop_df = copyLabelDropColumns(Monday_df)
#print(Mon_df_y[:5])
print(Mon_drop_df.shape)

#Tuesday Brute Force
BruteForce_df_y, BruteForce_drop_df = copyLabelDropColumns(Tuesday_BruteForce)
#print(BruteForce_df_y[:5])
print(BruteForce_drop_df.shape)

#Wednesday DOS
DOS_df_y, DOS_drop_df = copyLabelDropColumns(Wednesday_DOS)
#print(DOS_df_y[:5])
print(DOS_drop_df.shape)

#Thursday Web Attack
WebAttack_df_y, WebAttack_drop_df = copyLabelDropColumns(Thursday_WebAttack)
#print(WebAttack_df_y[:5])
print(WebAttack_drop_df.shape)

#Thursday Infiltration
Infiltration_df_y, Infiltration_drop_df = copyLabelDropColumns(Thursday_Infiltration)
#print(Infiltration_df_y[:5])
print(Infiltration_drop_df.shape)

#Friday Botnet
Botnet_df_y, Botnet_drop_df = copyLabelDropColumns(Friday_Botnet)
#print(Botnet_df_y[:5])
print(Botnet_drop_df.shape)

#Friday Portscan
Portscan_df_y, Portscan_drop_df = copyLabelDropColumns(Friday_Portscan)
#print(Portscan_df_y[:5])
print(Portscan_drop_df.shape)

#Friday DDOS
DDOS_df_y, DDOS_drop_df = copyLabelDropColumns(Friday_DDOS)
#print(DDOS_df_y[:5])
print(DDOS_drop_df.shape)

(529918, 65)
(445909, 65)
(692703, 65)
(170366, 65)
(288602, 65)
(191033, 65)
(286467, 65)
(225745, 65)


#### What columns are left

In [7]:
print(DDOS_drop_df.columns)

Index(['Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd URG Flags',
       'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s',
       'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length',
       'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance',
       'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count',
       'ACK Flag Count', 'URG Flag Count', 

#### Encode Labels

In [8]:
def encodeLabels(df):
    df['Label_encoding'] = df['Label'].apply(lambda x: 0 if x == 'BENIGN' else 1)
    df = df.drop(['Label'], axis=1)
    return df

Mon_df_y = encodeLabels(Mon_df_y)
#Make into series
Mon_df_y = Mon_df_y['Label_encoding'] 
#print(Mon_df_y[:5])

BruteForce_df_y = encodeLabels(BruteForce_df_y)
#Make into series
BruteForce_df_y = BruteForce_df_y['Label_encoding'] 
#print(BruteForce_df_y[:5])

DOS_df_y = encodeLabels(DOS_df_y)
#Make into series
DOS_df_y = DOS_df_y['Label_encoding'] 
#print(DOS_df_y[:5])

WebAttack_df_y = encodeLabels(WebAttack_df_y)
#Make into series
WebAttack_df_y = WebAttack_df_y['Label_encoding']
#print(WebAttack_df_y[:5])

Infiltration_df_y = encodeLabels(Infiltration_df_y)
#Make into series
Infiltration_df_y = Infiltration_df_y['Label_encoding']
#print(Infiltration_df_y[:5])

Botnet_df_y = encodeLabels(Botnet_df_y)
#Make into series
Botnet_df_y = Botnet_df_y['Label_encoding']
#print(Botnet_df_y[:5])

Portscan_df_y = encodeLabels(Portscan_df_y)
#Make into series
Portscan_df_y = Portscan_df_y['Label_encoding']
#print(Portscan_df_y[:5])

DDOS_df_y = encodeLabels(DDOS_df_y)
#Make into series
DDOS_df_y = DDOS_df_y['Label_encoding']
#print(DDOS_df_y[:5])


#### Check that correct labels are maintained

In [9]:
#print('Monday: ', '\n', Mon_df_y['Label_encoding'].value_counts())
#print('BruteForce:', '\n',  BruteForce_df_y['Label_encoding'].value_counts())
#print('DOS: ', '\n', DOS_df_y['Label_encoding'].value_counts())
#print('WebAttack: ', '\n', WebAttack_df_y['Label_encoding'].value_counts())
#print('Infiltration: ', '\n', Infiltration_df_y['Label_encoding'].value_counts())
#print('Botnet: ', '\n', Botnet_df_y['Label_encoding'].value_counts())
#print('Portscan: ', '\n', Portscan_df_y['Label_encoding'].value_counts())
#print('DDOS: ', '\n', DDOS_df_y['Label_encoding'].value_counts())

#### Train test split all data

In [10]:
#Monday train, test split
X_Mon_train, X_Mon_test, Y_Mon_train, Y_Mon_test = train_test_split(
    Mon_drop_df, Mon_df_y, 
    test_size=.2, 
    stratify=Mon_df_y, 
    random_state=28)

In [11]:
#BruteForce train, test split
X_BruteForce_train, X_BruteForce_test, Y_BruteForce_train, Y_BruteForce_test = train_test_split(
    BruteForce_drop_df, BruteForce_df_y, 
    test_size=.2, 
    stratify=BruteForce_df_y, 
    random_state=28)

In [12]:
#DOS train, test split
X_DOS_train, X_DOS_test, Y_DOS_train, Y_DOS_test = train_test_split(
    DOS_drop_df, DOS_df_y, 
    test_size=.2, 
    stratify=DOS_df_y, 
    random_state=28)

In [13]:
#WebAttack train, test split
X_WebAttack_train, X_WebAttack_test, Y_WebAttack_train, Y_WebAttack_test = train_test_split(
    WebAttack_drop_df, WebAttack_df_y, 
    test_size=.2, 
    stratify=WebAttack_df_y, 
    random_state=28)

In [14]:
#Infiltration train, test split
X_Infiltration_train, X_Infiltration_test, Y_Infiltration_train, Y_Infiltration_test = train_test_split(
    Infiltration_drop_df, Infiltration_df_y, 
    test_size=.2, 
    stratify=Infiltration_df_y, 
    random_state=28)

In [15]:
#Botnet train, test split
X_Botnet_train, X_Botnet_test, Y_Botnet_train, Y_Botnet_test = train_test_split(
    Botnet_drop_df, Botnet_df_y, 
    test_size=.2, 
    stratify=Botnet_df_y, 
    random_state=28)

In [16]:
#Portscan train, test split
X_Portscan_train, X_Portscan_test, Y_Portscan_train, Y_Portscan_test = train_test_split(
    Portscan_drop_df, Portscan_df_y, 
    test_size=.2, 
    stratify=Portscan_df_y, 
    random_state=28)


In [17]:
#Portscan train, test split
X_DDOS_train, X_DDOS_test, Y_DDOS_train, Y_DDOS_test = train_test_split(
    DDOS_drop_df, DDOS_df_y, 
    test_size=.2, 
    stratify=DDOS_df_y, 
    random_state=28)


#### Add Monday to all attack days

In [18]:
def addMonday(x_train, x_test, y_train, y_test):
    concat_x_train = pd.concat([X_Mon_train, x_train], ignore_index=True)
    concat_x_test = pd.concat([X_Mon_test, x_test], ignore_index=True)
    concat_y_train = pd.concat([Y_Mon_train, y_train], ignore_index=True)
    concat_y_test = pd.concat([Y_Mon_test, y_test], ignore_index=True)
    return concat_x_train, concat_x_test, concat_y_train, concat_y_test

In [19]:
#Concat BruteForce
BruteForceMon_x_train, BruteForceMon_x_test, BruteForceMon_y_train, BruteForceMon_y_test = addMonday(X_BruteForce_train, X_BruteForce_test, Y_BruteForce_train, Y_BruteForce_test)

#Concat DOS
DOSMon_x_train, DOSMon_x_test, DOSMon_y_train, DOSMon_y_test = addMonday(X_DOS_train, X_DOS_test, Y_DOS_train, Y_DOS_test)

#Concat WebAttack
WebAttackMon_x_train, WebAttackMon_x_test, WebAttackMon_y_train, WebAttackMon_y_test = addMonday(X_WebAttack_train, X_WebAttack_test, Y_WebAttack_train, Y_WebAttack_test)

#Concat Infiltration
InfiltrationMon_x_train, InfiltrationMon_x_test, InfiltrationMon_y_train, InfiltrationMon_y_test = addMonday(X_Infiltration_train, X_Infiltration_test, Y_Infiltration_train, Y_Infiltration_test)

#Concat Botnet
BotnetMon_x_train, BotnetMon_x_test, BotnetMon_y_train, BotnetMon_y_test = addMonday(X_Botnet_train, X_Botnet_test, Y_Botnet_train, Y_Botnet_test)

#Concat Portscan
PortscanMon_x_train, PortscanMon_x_test, PortscanMon_y_train, PortscanMon_y_test = addMonday(X_Portscan_train, X_Portscan_test, Y_Portscan_train, Y_Portscan_test)

#Concat DDOS
DDOSMon_x_train, DDOSMon_x_test, DDOSMon_y_train, DDOSMon_y_test = addMonday(X_DDOS_train, X_DDOS_test, Y_DDOS_train, Y_DDOS_test)

#### Scale features

In [20]:
def scaleFeatures(train, test):
    # Replace infinities with NaN
    train = train.replace([np.inf, -np.inf], np.nan)
    test  = test.replace([np.inf, -np.inf], np.nan)

    # Handle NaNs by filling with median of column
    train_medians = train.median()
    train = train.fillna(train_medians)
    test  = test.fillna(train_medians)

    #Declare scaler, fit on training
    scaler = MinMaxScaler().fit(train)

    #Transform train, test
    train_scaled = pd.DataFrame(scaler.transform(train), columns=train.columns)
    test_scaled  = pd.DataFrame(scaler.transform(test), columns=test.columns)

    return train_scaled, test_scaled

#Scale BruteForce
BruteForceMon_x_train, BruteForceMon_x_test = scaleFeatures(BruteForceMon_x_train, BruteForceMon_x_test)
#Scale DOS
DOSMon_x_train, DOSMon_x_test = scaleFeatures(DOSMon_x_train, DOSMon_x_test)
#Scale WebAttack
WebAttackMon_x_train, WebAttackMon_x_test = scaleFeatures(WebAttackMon_x_train, WebAttackMon_x_test)
#Scale Infiltration
InfiltrationMon_x_train, InfiltrationMon_x_test = scaleFeatures(InfiltrationMon_x_train, InfiltrationMon_x_test)
#Scale Botnet
BotnetMon_x_train, BotnetMon_x_test = scaleFeatures(BotnetMon_x_train, BotnetMon_x_test)
#Scale Portscan
PortscanMon_x_train, PortscanMon_x_test = scaleFeatures(PortscanMon_x_train, PortscanMon_x_test)
#Scale DDOS
DDOSMon_x_train, DDOSMon_x_test = scaleFeatures(DDOSMon_x_train, DDOSMon_x_test)

#### The Models

In [21]:
#Logistic Regression
logistic = LogisticRegression(max_iter=100)

scoring = {'AUC': 'roc_auc', 'Accuracy': 'accuracy', 'Average Precision': 'average_precision', 'F1': 'f1'}

param_grid_logistic = [
    # L2 (liblinear or lbfgs)
    {'penalty': ['l2'],
     'C': [0.01, 0.1, 1, 10],
     'solver': ['liblinear', 'lbfgs']},

    # L1 (liblinear only)
    {'penalty': ['l1'],
     'C': [0.01, 0.1, 1, 10],
     'solver': ['liblinear']},

    # No penalty (lbfgs)
    {'penalty': ['none'],
     'solver': ['lbfgs']}
]

# Setting refit='AUC', refits an estimator on the whole dataset with the
# parameter setting that has the best cross-validated AUC score.
# That estimator is made available at ``gs.best_estimator_`` along with
# parameters like ``gs.best_score_``, ``gs.best_params_`` and
# ``gs.best_index_``
grid_search_logistic = GridSearchCV(
    logistic, 
    param_grid=param_grid_logistic,
    refit='AUC',
    cv=5,
    n_jobs=-1,
    scoring=scoring
)


In [22]:
#KNN
knn = KNeighborsClassifier()
param_grid_knn = {
    'n_neighbors': [1, 3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [5, 10, 15, 20, 25, 30, 35, 40]
}
grid_search_knn = GridSearchCV(
    knn,
    param_grid=param_grid_knn,
    refit='AUC',
    cv=5,
    n_jobs=-1,
    scoring=scoring
)

In [23]:
#Random Forest Classifier
random = RandomForestClassifier(random_state=42, n_jobs=-1)

param_grid_rf = {
    'n_estimators': [100, 200, 500],       
    'max_depth': [None, 10, 20, 30],       
    'min_samples_split': [2, 5, 10],        
    'min_samples_leaf': [1, 2, 4],          
    'max_features': ['sqrt', 'log2', None]  
}

grid_search_rf = GridSearchCV(
    random,
    param_grid=param_grid_rf,
    refit='AUC',
    cv=5,
    n_jobs=-1,
    scoring=scoring   
)

In [24]:
#MLP Classifier
mlpclassifier = MLPClassifier(max_iter=100)

param_grid_mlp = {
   'hidden_layer_sizes': [(32,), (64,), (128,),(64, 32), (128, 64)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001, 0.001, 0.01],  
    'learning_rate': ['constant', 'adaptive'],
}

grid_search_mlp = GridSearchCV(
   mlpclassifier,
   param_grid=param_grid_mlp,
   refit='AUC',
   cv=5,
   n_jobs=-1,
   scoring=scoring
)

#### Brute Force Analysis

In [25]:
#Logistic Regression
grid_search_logistic.fit(BruteForceMon_x_train, BruteForceMon_y_train)
BF_logistic_bestestimator = grid_search_logistic.best_estimator_
BF_logistic_bestparams = grid_search_logistic.best_params_
BF_logistic_bestscore = grid_search_logistic.best_score_
print('Brute Force \n')
print('Logistic Regression \n')
print("Best parameters:", BF_logistic_bestparams)
print("Best cross-val score: {:.2f}%".format(BF_logistic_bestscore * 100))
print('Test score:')

#KNN
grid_search_knn.fit(BruteForceMon_x_train, BruteForceMon_y_train)
BF_knn_bestestimator = grid_search_knn.best_estimator_
BF_knn_bestparams = grid_search_knn.best_params_
BF_knn_bestscore = grid_search_knn.best_score_
print('KNN \n')
print("Best parameters for knn:", BF_knn_bestparams)
print("Best score for knn: {:.2f}%".format(BF_knn_bestscore * 100))
print('Test score:')

#Random Forest Classifier
grid_search_rf.fit(BruteForceMon_x_train, BruteForceMon_y_train)
BF_rf_bestestimator = grid_search_rf.best_estimator_
BF_rf_bestparams = grid_search_rf.best_params_
BF_rf_bestscore = grid_search_rf.best_score_
print('Random Forest \n')
print("Best parameters for random forest:", BF_rf_bestparams)
print("Best score for random forest: {:.2f}%".format(BF_rf_bestscore * 100))
print('Test score:')

#MLP Classifier
grid_search_mlp.fit(BruteForceMon_x_train, BruteForceMon_y_train)
BF_mlp_bestestimator = grid_search_mlp.best_estimator_
BF_mlp_bestparams = grid_search_mlp.best_params_
BF_mlp_bestscore = grid_search_mlp.best_score_
print('Multi-Layer Perceptron \n')
print("Best parameters for random forest:", BF_mlp_bestparams)
print("Best score for random forest: {:.2f}%".format(BF_mlp_bestscore * 100))
print('Test score:')

/opt/homebrew/Caskroom/miniforge/base/envs/cs135_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/homebrew/Caskroom/miniforge/base/envs/cs135_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://s

KeyboardInterrupt: 

#### DOS Analysis

#### WebAttack Analysis

#### Infiltration Analysis

#### Botnet Analysis

#### Portscan Analysis

#### DDOS Analysis